In [419]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [789]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DST = 4
K = 5

DATA = 0
KEYS = 1

In [581]:
#Set to create unique in house DataFrame 

tupOffense = [('', 'Game'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

tupDef = [('', 'Opp'),
          ('', 'Pts*'),
          ('Def', 'Sack'), 
          ('Def', 'Int'),
          ('Def', 'Saf'), 
          ('Def', 'FR'), 
          ('Def', 'Blk'), 
          ('Def', 'TD'), 
          ('Def', 'PA'),
          ('Def', 'PassYds/G'), 
          ('Def', 'RushYds/G'), 
          ('Def', 'TotYds/G')
         ]

tupKicker = [('', 'Game'), 
             ('', 'Pts*'),          
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

#Set to create unique in house DataFrame of master Dataframe that will have all players and seasons
masterTup = [('', 'PPR'),
             ('', 'Pos'),
             ('', 'Team'),
             ('', 'Bye'),
             ('', 'Pts'),
             ('', 'Week'),
             ('', 'Year'),
             ('', 'Game'),
             ('', 'Opp'),
             ('Passing', 'Att'),
             ('Passing', 'Cmp'),
             ('Passing', 'Yds'),
             ('Passing', 'TD'),
             ('Passing', 'Int'),
             ('Passing', '2Pt'),
             ('Rushing', 'Att'),
             ('Rushing', 'Yds'),
             ('Rushing', 'TD'),
             ('Rushing', '2Pt'),
             ('Receiving', 'Rec'),
             ('Receiving', 'Yds'),
             ('Receiving', 'TD'),
             ('Receiving', '2Pt'),
             ('Fumbles', 'FL'),
             ('Fumbles', 'TD'),
             ('Defense', 'Sack'),
             ('Defense', 'Int'),
             ('Defense', 'Saf'),
             ('Defense', 'FR'), 
             ('Defense', 'Blk'),
             ('Defense', 'TD'),
             ('Defense', 'PA'),
             ('Defense', 'PassYds/G'),
             ('Defense', 'RushYds/G'),
             ('Defense', 'TotYds/G'),
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [425]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [426]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    
    #Array [0] is where the data is stored. [1] stores the rules 
    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DST, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(K, pd.io.html.read_html(url_kicker_season17)[0]) 

In [427]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3
    index = nameIndex

    for i in data:
        index.append(i)


    size = len(index)

    for i in range(size):
        a1 = index[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        index[i] = a1[:count]
    
    return index

In [428]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [429]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [430]:
def setDataFrame(dframe, dataType, index, fixXa0):
    nameIndex = []
    dataMatrix = []
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    if(fixXa0):
        nameIndex = nameFix(nameIndex, frameArray[dataType]['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DST):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == K):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    return(dframe)



In [696]:
def createIndex(keyList):
    position = keyList['Pos'] 
    index = []
    
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        index = pd.MultiIndex.from_tuples(tupOffense)
    elif(position == 'K'):
        index = pd.MultiIndex.from_tuples(tupKicker)
    elif(position == 'DST'):
        index = pd.MultiIndex.from_tuples(tupDef)
        
    return index

In [804]:
def pullDataWeb():
    ffb_year = [i for i in range(2010, 2019)]
    ffb_stdscore = [1,0]
    ffb_week = [i for i in range(1,18)]
    ffb_pos = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']
    frameArray = []

    ffb_d1 = {'StdScore': ffb_stdscore, 'Year':ffb_year, 'Week': ffb_week, 'Pos': ffb_pos}

    keyList = []
    count = 0
    url = []


    #Create url string for every week. Save scoring, year, week, and pos for future reference
    for scoring in ffb_d1['StdScore']:
        for year in ffb_d1['year']:
            for week in ffb_d1['week']:
                for pos in ffb_d1['pos']:
                    url.append("https://www.footballdb.com/fantasy-football/index.html?pos={p}&yr={y}&wk={w}&rules={r}"\
                    .format(p=pos, y=year, w=week, r=scoring))
                    keylist.append({'stdscore':scoring, 'year':year, 'week':week, 'pos':pos})
                    
    #frame array will be a list of tuples. first value of tuple is dframe. second is
    #dictionary that contains the keys
    for i in url:
        framearray.append((pd.io.html.read_html(i)[0], keylist[count]))
        count += 1 

In [1134]:
def setDataFrame_weeks(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos'] #Get the position of the player
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): ('No' if keyList['StdScore'] else 'Yes'),
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    #Fix the names for QB, WR, TE, RB
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        retframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
        dname = dict()
        count = 0
        retframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))               
                               
        for i in nameIndex:
            dname[count] = i
            count += 1
        #Rename the frame and change the Pts* to Pts
        retframe = (retframe.rename(dname, axis = 'index'))
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
        
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [1340]:
def setDataFrame_test(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos'] #Get the positionof the data
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): ('No' if keyList['StdScore'] else 'Yes'),
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    

    
    #For QB, WR, TE, and RB, rename the columns and index to be more legible
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        
        d1 = dict(zip(dframe.columns.levels[0], 
            ['Passing', 'Passing','Passing', 'Passing', 'Rushing',\
             'Rushing', 'Rushing', 'Receiving',\
             'Receiving', 'Receiving', 'Receiving',\
             'Fumbles', 'Fumbles', 'Passing',\
             'Passing', 'Rushing', '']))
        
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        retframe = dframe.rename(columns=d1, level=0)
        
        d2 = dict(zip(retframe[('','Player')], nameIndex))
        retframe = retframe.set_index(('','Player'))
        retframe = retframe.rename_axis('')
        retframe = retframe.rename(d2, axis='index')
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')

    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
        dname = dict()
        count = 0
        retframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))               
                               
        for i in nameIndex:
            dname[count] = i
            count += 1
        #Rename the frame and change the Pts* to Pts
        retframe = (retframe.rename(dname, axis = 'index'))
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
        
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [1346]:
#MAIN STARTS HERE
frameArray = []

columns = pd.MultiIndex.from_tuples(masterTup)
frameMaster = DataFrame(columns = columns)
test_count = 0

for tup in tframeArray:
    index = createIndex(tup[KEYS])
    dframe = setDataFrame_test(tup[DATA], tup[KEYS], index)
    frameArray.append(dframe)
    frameMaster = pd.concat([frameMaster, dframe])
    
    test_count += 1
    if test_count == 1:
        break



In [1347]:
frameMaster

Defense  \
                  Bye     Game  Opp PPR Pos   Pts Team Week  Year     Blk   
Peyton Manning    NaN  IND@HOU  NaN  No  QB  35.0  NaN    1  2010     NaN   
Tom Brady         NaN   CIN@NE  NaN  No  QB  28.0  NaN    1  2010     NaN   
David Garrard     NaN  DEN@JAX  NaN  No  QB  25.0  NaN    1  2010     NaN   
Jay Cutler        NaN  DET@CHI  NaN  No  QB  25.0  NaN    1  2010     NaN   
Michael Vick      NaN   GB@PHI  NaN  No  QB  23.0  NaN    1  2010     NaN   
Philip Rivers     NaN    SD@KC  NaN  No  QB  23.0  NaN    1  2010     NaN   
Carson Palmer     NaN   CIN@NE  NaN  No  QB  23.0  NaN    1  2010     NaN   
Eli Manning       NaN  CAR@NYG  NaN  No  QB  22.0  NaN    1  2010     NaN   
Matt Hasselbeck   NaN   SF@SEA  NaN  No  QB  22.0  NaN    1  2010     NaN   
Vince Young       NaN  OAK@TEN  NaN  No  QB  20.0  NaN    1  2010     NaN   
Josh Freeman      NaN   CLE@TB  NaN  No  QB  20.0  NaN    1  2010     NaN   
Tony Romo         NaN  DAL@WAS  NaN  No  QB  17.0  NaN    1  2010     NaN   
Kyle Orton        NaN  DEN@JAX  NaN  No  QB  16.0  NaN    1  2010     NaN   
Derek Anderson    NaN  ARI@STL  NaN  No  QB  16.0  NaN    1  2010     NaN   
Aaron Rodgers     NaN   GB@PHI  NaN  No  QB  15.0  NaN    1  2010     NaN   
Drew Brees        NaN   MIN@NO  NaN  No  QB  15.0  NaN    1  2010     NaN   
Jason Campbell    NaN  OAK@TEN  NaN  No  QB  13.0  NaN    1  2010     NaN   
Trent Edwards     NaN  MIA@BUF  NaN  No  QB  12.0  NaN    1  2010     NaN   
Jake Delhomme     NaN   CLE@TB  NaN  No  QB  11.0  NaN    1  2010     NaN   
Brett Favre       NaN   MIN@NO  NaN  No  QB  10.0  NaN    1  2010     NaN   
Sam Bradford      NaN  ARI@STL  NaN  No  QB  10.0  NaN    1  2010     NaN   
Matt Schaub       NaN  IND@HOU  NaN  No  QB   8.0  NaN    1  2010     NaN   
Matt Ryan         NaN  ATL@PIT  NaN  No  QB   8.0  NaN    1  2010     NaN   
Matt Cassel       NaN    SD@KC  NaN  No  QB   8.0  NaN    1  2010     NaN   
Matt Moore        NaN  CAR@NYG  NaN  No  QB   7.0  NaN    1  2010     NaN   
Donovan McNabb    NaN  DAL@WAS  NaN  No  QB   7.0  NaN    1  2010     NaN   
Chad Henne        NaN  MIA@BUF  NaN  No  QB   7.0  NaN    1  2010     NaN   
Dennis Dixon      NaN  ATL@PIT  NaN  No  QB   7.0  NaN    1  2010     NaN   
Joe Flacco        NaN  BAL@NYJ  NaN  No  QB   6.0  NaN    1  2010     NaN   
Alex Smith        NaN   SF@SEA  NaN  No  QB   5.0  NaN    1  2010     NaN   
Matthew Stafford  NaN  DET@CHI  NaN  No  QB   2.0  NaN    1  2010     NaN   
Mark Sanchez      NaN  BAL@NYJ  NaN  No  QB   2.0  NaN    1  2010     NaN   
Seneca Wallace    NaN   CLE@TB  NaN  No  QB   0.0  NaN    1  2010     NaN   
Tim Tebow         NaN  DEN@JAX  NaN  No  QB   0.0  NaN    1  2010     NaN   
Kevin Kolb        NaN   GB@PHI  NaN  No  QB   0.0  NaN    1  2010     NaN   
Shaun Hill        NaN  DET@CHI  NaN  No  QB   0.0  NaN    1  2010     NaN   
Jimmy Clausen     NaN  CAR@NYG  NaN  No  QB   0.0  NaN    1  2010     NaN   

                 ...  Passing      Receiving            Rushing              
                 ...       TD  Yds       2Pt Rec TD Yds     2Pt Att TD  Yds  
Peyton Manning   ...        3  433         0   0  0   0       0   0  0    0  
Tom Brady        ...        3  258         0   0  0   0       0   0  0    0  
David Garrard    ...        3  170         0   0  0   0       0   7  0   10  
Jay Cutler       ...        2  372         0   0  0   0       0   5  0   22  
Michael Vick     ...        1  175         0   0  0   0       0  11  0  103  
Philip Rivers    ...        2  298         0   0  0   0       0   3  0    5  
Carson Palmer    ...        2  345         0   0  0   0       0   4  0    9  
Eli Manning      ...        3  263         0   0  0   0       0   4  0   -2  
Matt Hasselbeck  ...        2  170         0   0  0   0       0   2  1    4  
Vince Young      ...        2  154         0   0  0   0       0   7  0   30  
Josh Freeman     ...        2  182         0   0  0   0       0   2  0   34  
Tony Romo        ...        1  282         0   0

In [1345]:
dframe.index

Index(['Peyton Manning', 'Tom Brady', 'David Garrard', 'Jay Cutler',
       'Michael Vick', 'Philip Rivers', 'Carson Palmer', 'Eli Manning',
       'Matt Hasselbeck', 'Vince Young', 'Josh Freeman', 'Tony Romo',
       'Kyle Orton', 'Derek Anderson', 'Aaron Rodgers', 'Drew Brees',
       'Jason Campbell', 'Trent Edwards', 'Jake Delhomme', 'Brett Favre',
       'Sam Bradford', 'Matt Schaub', 'Matt Ryan', 'Matt Cassel', 'Matt Moore',
       'Donovan McNabb', 'Chad Henne', 'Dennis Dixon', 'Joe Flacco',
       'Alex Smith', 'Matthew Stafford', 'Mark Sanchez', 'Seneca Wallace',
       'Tim Tebow', 'Kevin Kolb', 'Shaun Hill', 'Jimmy Clausen'],
      dtype='object', name='')

In [1320]:
dframe[('','Player')]


        d1 = dict(zip(dframe.columns.levels[0], 
            ['Passing', 'Passing','Passing', 'Passing', 'Rushing',\
             'Rushing', 'Rushing', 'Receiving',\
             'Receiving', 'Receiving', 'Receiving',\
             'Fumbles', 'Fumbles', 'Passing',\
             'Passing', 'Rushing', '']))

0                               hi
1                Tom BradyT. Brady
2          David GarrardD. Garrard
3              Jay CutlerJ. Cutler
4              Michael VickM. Vick
5           Philip RiversP. Rivers
6           Carson PalmerC. Palmer
7            Eli ManningE. Manning
8     Matt HasselbeckM. Hasselbeck
9              Vince YoungV. Young
10          Josh FreemanJ. Freeman
11                Tony RomoT. Romo
12              Kyle OrtonK. Orton
13       Derek AndersonD. Anderson
14         Aaron RodgersA. Rodgers
15              Drew BreesD. Brees
16       Jason CampbellJ. Campbell
17         Trent EdwardsT. Edwards
18        Jake DelhommeJ. Delhomme
19             Brett FavreB. Favre
20         Sam BradfordS. Bradford
21            Matt SchaubM. Schaub
22                Matt RyanM. Ryan
23            Matt CasselM. Cassel
24              Matt MooreM. Moore
25         Donovan McNabbD. McNabb
26              Chad HenneC. Henne
27            Dennis DixonD. Dixon
28             Joe F

# TEST SECTION STARTS AFTER THIS LINE 

Try to extract 2010 QBs only

In [1120]:
frameMaster.filter(items = [('','Game'), ('', 'Bye')])
PPR = frameMaster[('','PPR')] == 'Yes'

In [1121]:
Year = frameMaster[('', 'Year')] == 2015

In [1122]:
Pos_QB = frameMaster[('', 'Pos')] == 'QB'
Week = frameMaster['', 'Week'] < 18

In [1123]:
df = frameMaster[PPR & Year & Pos_QB & Week]

In [1124]:
df

Empty DataFrame
Columns: [(, Bye), (, Game), (, Opp), (, PPR), (, Pos), (, Pts), (, Team), (, Week), (, Year), (Defense, Blk), (Defense, FR), (Defense, Int), (Defense, PA), (Defense, PassYds/G), (Defense, RushYds/G), (Defense, Sack), (Defense, Saf), (Defense, TD), (Defense, TotYds/G), (Fumbles, FL), (Fumbles, TD), (Kicking, 50+), (Kicking, FGA), (Kicking, FGM), (Kicking, XPA), (Kicking, XPM), (Passing, 2Pt), (Passing, Att), (Passing, Cmp), (Passing, Int), (Passing, TD), (Passing, Yds), (Receiving, 2Pt), (Receiving, Rec), (Receiving, TD), (Receiving, Yds), (Rushing, 2Pt), (Rushing, Att), (Rushing, TD), (Rushing, Yds)]
Index: []

[0 rows x 40 columns]

In [1117]:
df['Tom Brady']


KeyError: 'Tom Brady'

In [1047]:
week

Tom Brady              1
Marcus Mariota         1
Carson Palmer          1
Alex Smith             1
Tony Romo              1
Aaron Rodgers          1
Philip Rivers          1
Nick Foles             1
Andy Dalton            1
Andrew Luck            1
Ben Roethlisberger     1
Matt Ryan              1
Drew Brees             1
Jameis Winston         1
Russell Wilson         1
Tyrod Taylor           1
Matthew Stafford       1
Ryan Fitzpatrick       1
Jay Cutler             1
Sam Bradford           1
Ryan Tannehill         1
Cam Newton             1
Matt McGloin           1
Johnny Manziel         1
Brian Hoyer            1
Ryan Mallett           1
Blake Bortles          1
Colin Kaepernick       1
Kirk Cousins           1
Teddy Bridgewater      1
                      ..
Philip Rivers         17
Tyrod Taylor          17
A.J. McCarron         17
Blaine Gabbert        17
Drew Brees            17
Jameis Winston        17
Jay Cutler            17
Aaron Rodgers         17
Ryan Fitzpatrick      17


In [1048]:
d1 = {'Yds':yds, 'Week':week}

In [1062]:
d1 = DataFrame(d1)
d2 = d1.loc['Tom Brady']

d2.iloc[1][1]

'466'

In [1030]:
#sns.lmplot('Pts', 'Week', data=d2, fit_reg = True)